**Общая информация**

**Срок сдачи:** 20 марта 2019, 06:00 <br\>
**Штраф за опоздание:** -1 балл после 06:00 20 марта, -2 балла после 06:00 27 марта, -3 балла после 06:00 3 апреля
-4 балла после 06:00 10 апреля

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush

Необходимо в slack скинуть ссылку в личный чат @alkhamush
Пример ссылки:
https://github.com/stroykova/spheremailru/stroykova_hw1.ipynb 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

###### Задание 2 (1 балл)
Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy

###### Задание 3 (1 балл)
Для iris найдите такой параметр n_neighbors, при котором выдаётся наилучший score. Нарисуйте график зависимости score от n_neighbors

###### Задание 4 (1.5 балла)
Добавить algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris и mnist
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

###### Задание 5 (0.5 балла)
Описать для чего нужны следующие библиотеки/классы/функции (список будет ниже)

**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять структуру кода и текст. В противном случае -0.5 баллов


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata
from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
%load_ext pycodestyle_magic

In [ ]:
%%pycodestyle

class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin, ClassifierMixin):
    
    def __init__(self, n_neighbors, algorithm='brute'):
        self.n_neighbors = n_neighbors 
        self.algorithm = algorithm 
        pass
    
    def fit(self, X, y):
        self.X = X 
        self.y = y 
        pass
    
    def predict(self, X):
        y_pred = np.zeros(X.shape[0]) 
        for j in range(0, X.shape[0]): 
            dist = np.zeros(self.X.shape[0]) 
            for i in range(0, self.X.shape[0]): 
                dist[i] = np.sum((self.X[i][:] - X[j][:])**2) 
            kneigh = np.argpartition(dist, self.n_neighbors)[0:self.n_neighbors]
            unique_values, occur_counts = np.unique(self.y[kneigh], return_counts = True)
            y_pred[j] = unique_values[np.argmax(occur_counts)]
        return y_pred 
    
    def predict_proba(self, X): 
        unique_values = np.unique(self.y)
        y_pred = np.zeros((X.shape[0], unique_values.shape[0])) 
        for j in range(X.shape[0]): 
            dist = np.zeros(self.X.shape[0]) 
            for i in range(self.X.shape[0]): 
                dist[i] = np.sum((self.X[i][:] - X[j][:])**2) 
            kneigh = np.argpartition(dist, self.n_neighbors) 
            for i in range(unique_values.size):
                y_pred[j][i] = np.sum(self.y[kneigh[:self.n_neighbors]] == unique_values[i])/self.n_neighbors 
        return y_pred 
    
    def score(self, X, y):
        truth = np.zeros(y.size) 
        pred = self.predict(X) 
        for i in range(y.size): 
            if (pred[i] == y[i]): 
                truth[i] = 1 
        accuracy = np.sum(truth)/y.size 
        return accuracy

**IRIS**

In [ ]:
iris = datasets.load_iris()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 3

In [ ]:
score_table = np.zeros(y_test.size)
score_table[0] = 0
for i in range(1, y_test.size):
    my_clf = KNeighborsClassifier(n_neighbors=i, algorithm='brute')
    my_clf.fit(X_train, y_train)    score_table[i] = my_clf.score(X_test, y_test)
score_max = np.argmax(score_table)
score_max


In [ ]:
plt.plot(range(y_test.size), score_table)
plt.ylabel('score')
plt.xlabel('number of neighbors')
plt.show()

**MNIST**

In [ ]:
mnist = fetch_mldata('MNIST original')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 4

In [ ]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(mnist.data, mnist.target, test_size=0.01, stratify=mnist.target)

In [ ]:
%time clf.fit(X_train, y_train)

In [ ]:
%time my_clf.fit(X_train, y_train)

In [ ]:
%time clf.predict(X_test)

In [ ]:
%time my_clf.predict(X_test)

In [ ]:
%time clf.predict_proba(X_test)

In [ ]:
%time my_clf.predict_proba(X_test)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
my_clf.score(X_test, y_test)

Задание 5

In [ ]:
# seaborn - библиотека для рисования графиков
# matplotlib - библиотека для рисования графиков
# train_test_split - функция, разделяющая выборку на обучающую и тестовую
# Pipelin%load_ext e (from sklearn.pipeline import Pipeline) - класс, позволяющий применять последовательность преобразований к исходным данным
# StandardScaler (from sklearn.preprocessing import StandardScaler) - функция, преобразующая каждый столбец матрицы путём сдвига и масштабирования к форме, когда его среднее значение равно 0, а стандартное отклонение равно 1
# ClassifierMixin - класс mixin для всех классификаторов в scikit-learn
# NeighborsBase - базовый класс для поиска ближайших соседей
# KNeighborsMixin - класс mixin для поиска ближайших соседей
# SupervisedIntegerMixin - класс mixin для реализации обучения с учителем